<a href="https://colab.research.google.com/github/sourcecode369/TensorFlow-2.0/blob/master/tensorflow_2.0_docs/TensorFlow%20Core/Guide/Accelerators/GPU/TensorFlow_2_0_Accelerators_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install TensorFlow

In [3]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 86.3MB 1.3MB/s 
     |████████████████████████████████| 450kB 46.0MB/s 
     |████████████████████████████████| 3.8MB 33.2MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0rc3
    Uninstalling tensorflow-1.15.0rc3:
      Successfully uninstalled tensorflow-1.15.0rc3


### Import TensorFlow

In [0]:
from __future__ import absolute_import, print_function, division, unicode_literals
import tensorflow as tf

> This guide is for users who have tried these approaches and found that they need find-grained control of how TensorFlow uses the GPU.



In [2]:
print(tf.__version__)
print(tf.test.is_gpu_available())

2.0.0
False


In [3]:
print('Num of GPUs Available: ', len(tf.config.experimental.list_physical_devices("GPU")))

Num of GPUs Available:  0


### Logging device placement

In [4]:
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1, 2, 3],[4, 5, 6]])
b = tf.constant([[1, 2], [3, 4] ,[5, 6]])
c = tf.matmul(a, b)
print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


### Manual device placement

In [5]:
tf.debugging.set_log_device_placement(True)
with tf.device("/CPU:0"):
    a = tf.constant([[1,2,3],[4,5,6]])
    b = tf.constant([[1,2],[3,4],[5,6]])

c = tf.matmul(a, b)
print(c) 

tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


### Limiting GPU Memory Growth

In [0]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [0]:
try:
    gpus = tf.config.experimental.list_physical_devices("GPU")
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices("GPU")
            print(len(gpus), "Physical GPUs", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
except:
    print("No GPUS Found")

In [0]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), len(logical_gpus))
    except:
        print(e)

### Using a single GPU on a multi-GPU system

In [12]:
tf.debugging.set_log_device_placement(True)

try:
    with tf.device('/device:GPU:2'):
        a = tf.constant([[1,2,3],[4,5,6]])
        b = tf.constant([[1,2],[3,4],[5,6]])
        c = tf.matmul(a, b)
except RuntimeError as e:
    print(e)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0


In [13]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

a = tf.constant([[1,2,3],[4,5,6]])
b = tf.constant([[1,2],[3,4],[5,6]])
c = tf.matmul(a, b)
print(c)


tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


### Using multiple GPUs

In [20]:
!pip install --upgrade tensorflow
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.0.0)
TensorFlow version:  2.0.0


In [21]:

tf.debugging.set_log_device_placement(True)
tf.config.set_soft_device_placement(True)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024),
             tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        )
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print("Physical GPUs", len(gpus), "Logical GPUs", len(logical_gpus))
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs found.")


No GPUs found.


In [0]:
# tf.debugging.set_log_device_placement(True)
# tf.config.set_soft_device_placement(True)

# mirrorerd_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice)
# with mirrorerd_strategy.scope():
#     inputs = tf.keras.layers.Input(shape=(1,))
#     predictions = tf.keras.layers.Dense(1)(inputs)
#     model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
#     model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["accuracy"])

#### Manual Placement

In [0]:
tf.debugging.set_log_device_placement(True)
tf.config.set_soft_device_placement(True)

gpus = tf.config.experimental.list_logical_devices("GPU")
if gpus:
    c = []
    for gpu in gpus:
        with tf.device(gpu.name):
            a = tf.constant([[1,2,3],[4,5,6]])
            b = tf.constant([[1,2],[3,4],[5,6]])
    with tf.device("/CPU:0"):
        matmul_sum = tf.add_n(c)

    print(matmul_sum)